# Phase-1 Testing

## Required Task:
- Split large Pdf to single page pdf and store it in local database
- Convert large pdf to single page image store ionto local database
- Convert Image to text without preprocessing and store it on text file
- Convert Image to text after preprocesing and store it on same text file and return the file

In [1]:
## Importing lib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
plt.style.use('dark_background')


import os
import shutil
import fitz
import PyPDF2
import sys
import cv2
import pytesseract as pyt
from PIL import Image
from tqdm import tqdm

In [3]:

## Image Read and Preprocessing

def readCV(path):

    imageRAW = cv2.imread(path, cv2.IMREAD_UNCHANGED)

    return imageRAW

## Basic Health of the image
def healthCheck(image):

    print('Here is the Basic Health Check of the Image')
    display('Type of the Image is:',type(image))
    display('Datatype of the Image:',image.dtype)
    display('Shape of the Image:',image.shape)
    
## Gray Scale conversion of the Image
def gray(image):

    ## Approach-2
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imageGray

## Binary Thresolding
def binThres(image):

    ## Threshold the image to create a binary image  using Otsu’s Binarization

    Thresh, imgBinThres = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return imgBinThres

 ## Resizing with scale factor
def resizeScaling(image):

    ## As cv2 shape come up with height and width format
    height, width = image.shape[:2]
    scale_factor =  2.0        # float(input('Put the Scale Factor :')) 

    ## Resize the image as per the scale factpr
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    new_size = (new_width, new_height)

    # Resize the image using the new dimensions
    imgScaled = cv2.resize(image, new_size, fx = 1.2, fy = 1.5, interpolation = cv2.INTER_LINEAR)

    ## fx: A scaling factor along the horizontal axis.
    ## fy: A scaling factor along the vertical axis.

    return imgScaled


# Denoising of image saving it into dst image 
def noiseRem(image):

    imgDenoise1 = cv2.fastNlMeansDenoising(image, None, 10, 7, 21)

    return imgDenoise1

In [ ]:
# ## function to split pdf into single pages and images & store it into different database

# def pdfSplitExtract(input_file_path, output_pdf_folder_path, output_image_folder_path):
    
#     ## Step-1 : Checking and creating existance of output main path and directory

#     ## Write the name of the folder
#     pdf_output_folder_name = 'splitted_pdf_files'
#     image_output_folder_name = 'splitted_image_txt_files'

#     ## For pdf
#     isExisting_pdf = os.path.exists(output_pdf_folder_path)             #Checking if the pdf dir exists 

#     if not isExisting_pdf:              # If the pdf path doesn`t exist then create the path and dir                        
#         os.mkdir(f"./All_pdf_image_txt/{pdf_output_folder_name}/")

#     ## For Image
#     isExisting_img = os.path.exists(output_image_folder_path)           #Checking if the image dir exists 

#     if not isExisting_img:              # If the path doesn`t exist then create the path and dir                         
#         os.mkdir(f"./All_pdf_image_txt/{image_output_folder_name}/")

#     print(' *********  Step-1 : Output PDF and IMAGE Base Folder Checking , Removal and Creating completed successfully ********* ')

#     ## Step-2: Creating Current file named directory
    
#     ## Extract file name 

#     filename = os.path.basename(input_file_path)   # Split the input filename from the full path

#     current_pdf_dir_name = os.path.splitext(filename)[0]   ## os.path.splitext(filename) will split text and will return tuple  
    
#     ## PDF output store

#     # Create current pdf file name directory
#     output_pdf_file_path = os.path.join(output_pdf_folder_path, current_pdf_dir_name)

#     # Remove same name folder if it exists
#     if os.path.exists(output_pdf_file_path):
#         shutil.rmtree(output_pdf_file_path)

#     # create the new current pdf named directory if it doesn't exist
#     if not os.path.exists(output_pdf_file_path):
#         os.mkdir(output_pdf_file_path)

#     ## Image output store

#     # Create current image file name directory
#     output_image_file_path = os.path.join(output_image_folder_path, current_pdf_dir_name)

#     # Remove same name folder if it exists
#     if os.path.exists(output_image_file_path):
#         shutil.rmtree(output_image_file_path)

#     # create the new directory if it doesn't exist
#     if not os.path.exists(output_image_file_path):
#         os.mkdir(output_image_file_path)

#     print(' *********  Step-2 : Output Sub/Current File Named Folder Checking , Removal and Creating completed successfully ********* ')

#     ## Step-3: Split whole pdf into multiple single pdf pages and store it into the pdf named folder

#     # Open the PDF file in read-binary mode
#     with open(input_file_path, 'rb') as file:

#         # Create a PDF reader object
#         pdf_reader = PyPDF2.PdfReader(file)
#         # print(pdf_reader)

#         # Get the total number of pages in the PDF file
#         num_pages = len(pdf_reader.pages)
#         # print(num_pages)

#         # Loop through each page of the PDF file
#         for page_num in range(num_pages):
#             # print(page_num)
            
#             # Showing each page
#             page = pdf_reader.pages[page_num]
        
#             # Create a new PDF writer object
#             pdf_writer = PyPDF2.PdfWriter()

#             # Add the current page to the new PDF writer object
#             pdf_writer.add_page(page)

        
#             ## Save the current page as a new PDF file with a name that includes the page number
#             final_output_pdf_files = os.path.join(output_pdf_file_path, f'{current_pdf_dir_name}_page_{page_num+1}.pdf')

#             # Remove the old pdf files with same name if it exists
#             if os.path.exists(final_output_pdf_files):
#                 os.remove(final_output_pdf_files)

#             with open(final_output_pdf_files, 'wb') as output_pdf_file:
#                 pdf_writer.write(output_pdf_file)


#         print(" *********  Step-3 : PDF to Single PDF Pages done Successfully ********* ")

#         ## Step-4: Split whole pdf into multiple single images of each pages and store it into the pdf named folder

#         # Open the PDF
#         pdf_doc = fitz.open(input_file_path)
#         format='JPEG'
#         dpi = 200
#         # Iterate over the pages
#         for page_idx in range(pdf_doc.page_count):
            
#             # Get the current page
#             page = pdf_doc[page_idx]
#             # Convert the page to an RGB image
#             pix = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
#             image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
#             # Set the output file name
            
#             final_output_image_files = os.path.join(output_image_file_path, f"{current_pdf_dir_name}_page{page_idx+1}.{format.lower()}")

#             # Remove the old image files with same name if it exists
#             if os.path.exists(final_output_image_files):
#                 os.remove(final_output_image_files)

#             # Save the image to disk
#             image.save(final_output_image_files, format=format.lower())      ## also we can write image.writeJPEG(file_name) or image.writePNG(file_name)

#         # # Close the PDF document and image
#         # pdf_doc.close()
#         # image.close()
#         print(" *********  Step-4 : PDF to Single Page Image Conversion done Successfully ********* ")

#         ## Step-5: Convert Image to text without and with preprocessing and store it on text file

#         ## Creating output text file & path where extracted text will be printed
#         output_file = os.path.join(output_image_file_path, f'Extracted_text_{current_pdf_dir_name}.txt')

#         # Check if the text file already exists
#         if os.path.exists(output_file):
#             os.remove(output_file)
#             print(f"Deleted existing text file named {output_file}")

#         ## Get access of list of ONLY image file paths in folder
#         # image_file_list = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpeg'))]
#         image_file_list = [os.path.join(output_image_file_path, f) for f in os.listdir(output_image_file_path) 
#                            if os.path.isfile(os.path.join(output_image_file_path, f)) and f.lower().endswith(('.jpg', '.jpeg', '.png'))]
#         # print(image_file_list)

#         with open(output_file, 'w') as file:
#             for image_file in image_file_list:

#                 # image_path = os.path.join(input_folder, image_file)
#                 imgRaw = readCV(image_file)
#                 # healthCheck(imgRaw)

#                 ## Without Preprocessing
#                 text_wo = pyt.image_to_string(imgRaw)
#                 file.write(f"\n\n ******************************** Text extracted from {image_file} Before Preprocessing *********************************** \n\n{text_wo}\n\n")
                
#                 ## Preprocessing
#                 imgGray = gray(imgRaw)
#                 imgBinTh = binThres(imgGray)
#                 imgScaled = resizeScaling(imgBinTh)
#                 imgDenoise = noiseRem(imgScaled)

#                 text_w = pyt.image_to_string(imgDenoise)
#                 file.write(f"\n\n ******************************** Text extracted from {image_file} After Preprocessing *********************************** \n\n{text_w}\n\n")

#         print(" *********  Step-5 : Image to Text Extraction without and With Preprocessing done Successfully ********* ")

        

In [3]:
# ## function to split pdf into single pages and images & store it into different database

# def pdfSplitExtract(input_file_path, output_pdf_folder_path, output_image_folder_path):
    
#     ## Step-1 : Checking and creating existance of output main path and directory

#     ## Write the name of the folder
#     pdf_output_folder_name = 'splitted_pdf_files'
#     image_output_folder_name = 'splitted_image_txt_files'

#     ## For pdf
#     isExisting_pdf = os.path.exists(output_pdf_folder_path)             #Checking if the pdf dir exists 

#     if not isExisting_pdf:              # If the pdf path doesn`t exist then create the path and dir                        
#         os.mkdir(f"./All_pdf_image_txt/{pdf_output_folder_name}/")

#     ## For Image
#     isExisting_img = os.path.exists(output_image_folder_path)           #Checking if the image dir exists 

#     if not isExisting_img:              # If the path doesn`t exist then create the path and dir                         
#         os.mkdir(f"./All_pdf_image_txt/{image_output_folder_name}/")

#     print(' *********  Step-1 : Output PDF and IMAGE Base Folder Checking , Removal and Creating completed successfully ********* ')

#     ## Step-2: Creating Current file named directory
    
#     ## Extract file name 

#     filename = os.path.basename(input_file_path)   # Split the input filename from the full path

#     current_pdf_dir_name = os.path.splitext(filename)[0]   ## os.path.splitext(filename) will split text and will return tuple  
    
#     ## PDF output store

#     # Create current pdf file name directory
#     output_pdf_file_path = os.path.join(output_pdf_folder_path, current_pdf_dir_name)

#     # Remove same name folder if it exists
#     if os.path.exists(output_pdf_file_path):
#         shutil.rmtree(output_pdf_file_path)

#     # create the new current pdf named directory if it doesn't exist
#     if not os.path.exists(output_pdf_file_path):
#         os.mkdir(output_pdf_file_path)

#     ## Image output store

#     # Create current image file name directory
#     output_image_file_path = os.path.join(output_image_folder_path, current_pdf_dir_name)

#     # Remove same name folder if it exists
#     if os.path.exists(output_image_file_path):
#         shutil.rmtree(output_image_file_path)

#     # create the new directory if it doesn't exist
#     if not os.path.exists(output_image_file_path):
#         os.mkdir(output_image_file_path)

#     print(' *********  Step-2 : Output Sub/Current File Named Folder Checking , Removal and Creating completed successfully ********* ')

#     ## Step-3: Split whole pdf into multiple single pdf pages and store it into the pdf named folder

#     # Open the PDF file in read-binary mode
#     with open(input_file_path, 'rb') as file:

#         # Create a PDF reader object
#         pdf_reader = PyPDF2.PdfReader(file)
#         # print(pdf_reader)

#         # Get the total number of pages in the PDF file
#         num_pages = len(pdf_reader.pages)
#         # print(num_pages)

#         # Loop through each page of the PDF file
#         for page_num in range(num_pages):
#             # print(page_num)
            
#             # Showing each page
#             page = pdf_reader.pages[page_num]
        
#             # Create a new PDF writer object
#             pdf_writer = PyPDF2.PdfWriter()

#             # Add the current page to the new PDF writer object
#             pdf_writer.add_page(page)

        
#             ## Save the current page as a new PDF file with a name that includes the page number
#             final_output_pdf_files = os.path.join(output_pdf_file_path, f'{current_pdf_dir_name}_page_{page_num+1}.pdf')

#             # Remove the old pdf files with same name if it exists
#             if os.path.exists(final_output_pdf_files):
#                 os.remove(final_output_pdf_files)

#             with open(final_output_pdf_files, 'wb') as output_pdf_file:
#                 pdf_writer.write(output_pdf_file)


#         print(" *********  Step-3 : PDF to Single PDF Pages done Successfully ********* ")

#         ## Step-4: Split whole pdf into multiple single images of each pages and store it into the pdf named folder

#         # Open the PDF
#         pdf_doc = fitz.open(input_file_path)
#         format='JPEG'
#         dpi = 200
#         # Iterate over the pages
#         for page_idx in range(pdf_doc.page_count):
            
#             # Get the current page
#             page = pdf_doc[page_idx]
#             # Convert the page to an RGB image
#             pix = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
#             image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
#             # Set the output file name
            
#             final_output_image_files = os.path.join(output_image_file_path, f"{current_pdf_dir_name}_page{page_idx+1}.{format.lower()}")

#             # Remove the old image files with same name if it exists
#             if os.path.exists(final_output_image_files):
#                 os.remove(final_output_image_files)

#             # Save the image to disk
#             image.save(final_output_image_files, format=format.lower())      ## also we can write image.writeJPEG(file_name) or image.writePNG(file_name)

#         # # Close the PDF document and image
#         # pdf_doc.close()
#         # image.close()
#         # print(" *********  Step-4 : PDF to Single Page Image Conversion done Successfully ********* ")

#         # ## Step-5: Convert Image to text without and with preprocessing and store it on text file

#         # ## Creating output text file & path where extracted text will be printed
#         # output_file = os.path.join(output_image_file_path, f'Extracted_text_{current_pdf_dir_name}.txt')

#         # # Check if the text file already exists
#         # if os.path.exists(output_file):
#         #     os.remove(output_file)
#         #     print(f"Deleted existing text file named {output_file}")

#         # ## Get access of list of ONLY image file paths in folder
#         # # image_file_list = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpeg'))]
#         # image_file_list = [os.path.join(output_image_file_path, f) for f in os.listdir(output_image_file_path) 
#         #                    if os.path.isfile(os.path.join(output_image_file_path, f)) and f.lower().endswith(('.jpg', '.jpeg', '.png'))]
#         # # print(image_file_list)

#         # with open(output_file, 'w') as file:
#         #     for image_file in image_file_list:

#         #         # image_path = os.path.join(input_folder, image_file)
#         #         imgRaw = readCV(image_file)
#         #         # healthCheck(imgRaw)

#         #         ## Without Preprocessing
#         #         text_wo = pyt.image_to_string(imgRaw)
#         #         file.write(f"\n\n ******************************** Text extracted from {image_file} Before Preprocessing *********************************** \n\n{text_wo}\n\n")
                
#         #         ## Preprocessing
#         #         imgGray = gray(imgRaw)
#         #         imgBinTh = binThres(imgGray)
#         #         imgScaled = resizeScaling(imgBinTh)
#         #         imgDenoise = noiseRem(imgScaled)

#         #         text_w = pyt.image_to_string(imgDenoise)
#         #         file.write(f"\n\n ******************************** Text extracted from {image_file} After Preprocessing *********************************** \n\n{text_w}\n\n")

#         # print(" *********  Step-5 : Image to Text Extraction without and With Preprocessing done Successfully ********* ")

        

In [13]:

# user_input = str(input("Write the file name: "))
# # input_file_path = './All_pdf_image_txt/test_pdf_full/' + user_input + '.pdf'

# input_file_path_ami = './All_pdf_image_txt/test_Ami/' + user_input + '.pdf'

# output_pdf_folder_path = './All_pdf_image_txt/splitted_pdf_files/ami_test'
# output_image_folder_path = './All_pdf_image_txt/splitted_image_txt_files/ami_test_txt'

# ## Call the function

# tqdm(pdfSplitExtract(input_file_path_ami, output_pdf_folder_path, output_image_folder_path))

 *********  Step-1 : Output PDF and IMAGE Base Folder Checking , Removal and Creating completed successfully ********* 
 *********  Step-2 : Output Sub/Current File Named Folder Checking , Removal and Creating completed successfully ********* 
 *********  Step-3 : PDF to Single PDF Pages done Successfully ********* 


## Test_Experiment:
- Flow:
        - pdf --> image conversion --> image processing  --> create and save into new folder